In [11]:
import datetime
import glob
import os
import sys
import shutil
import pickle
import random
import numpy as np
import tensorflow as tf
from projlearn import *

In [32]:
"""
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string( 'model',  'baseline', 'Model name.')
flags.DEFINE_string( 'train', 'train.npz', 'Training set.')
flags.DEFINE_string( 'test',   'test.npz', 'Test set.')
flags.DEFINE_float(  'stddev',        .01, 'Value of stddev for matrix initialization.')
flags.DEFINE_float(  'lambdac',       .10, 'Value of lambda.')
flags.DEFINE_integer('seed',          1337, 'Random seed.')
flags.DEFINE_integer('num_epochs',    700, 'Number of training epochs.')
flags.DEFINE_integer('batch_size',   1024, 'Batch size.')
flags.DEFINE_boolean('gpu',          False, 'Try using GPU.')
"""
# Parameters
model_type = 'regularized_synonym_phi'
#model_type = 'baseline'
train_vectors = 'train.npz'
test_vectors = 'test.npz'
stddev = 0.1
lambdac = 0.1
#lambdac = 0
#seed = 1337
seed = 1000
num_epochs = 700
batch_size = 1024
gpu = False

w2v = False

In [28]:
def train(model, data, callback=lambda: None):
    train_op = tf.train.AdamOptimizer(epsilon=1.).minimize(model.loss)

    train_losses, test_losses = [], []
    train_times = []

    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

        feed_dict_train, feed_dict_test = {
            model.X: data.X_train,
            model.Y: data.Y_train,
            model.Z: data.Z_train
        }, {
            model.X: data.X_test,
            model.Y: data.Y_test,
            model.Z: data.Z_test
        }

        steps = max(data.Y_train.shape[0] // batch_size, 1)

        print('Cluster %d: %d train items and %d test items available; using %d steps of %d items.' % (
            data.cluster + 1,
            data.X_train.shape[0],
            data.X_test.shape[0],
            steps,
            min(batch_size, data.X_train.shape[0])),
        flush=True)

        for epoch in range(num_epochs):
            X, Y, Z = data.train_shuffle()

            for step in range(steps):
                head =  step      * batch_size
                tail = (step + 1) * batch_size

                feed_dict = {
                    model.X: X[head:tail, :],
                    model.Y: Y[head:tail, :],
                    model.Z: Z[head:tail, :]
                }

                t_this = datetime.datetime.now()
                sess.run(train_op, feed_dict=feed_dict)
                t_last = datetime.datetime.now()

                train_times.append(t_last - t_this)

            if (epoch + 1) % 10 == 0 or (epoch == 0):
                train_losses.append(sess.run(model.loss, feed_dict=feed_dict_train))
                test_losses.append(sess.run(model.loss,  feed_dict=feed_dict_test))

                print('Cluster %d: epoch = %05d, train loss = %f, test loss = %f.' % (
                    data.cluster + 1,
                    epoch + 1,
                    train_losses[-1] / data.X_train.shape[0],
                    test_losses[-1]  / data.X_test.shape[0]),
                file=sys.stderr, flush=True)

        t_delta = sum(train_times, datetime.timedelta())
        print('Cluster %d done in %s.' % (data.cluster + 1, str(t_delta)), flush=True)
        callback(sess)

        return sess.run(model.Y_hat, feed_dict=feed_dict_test)


In [29]:
random.seed(seed)
tf.set_random_seed(seed)

if not gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = ''

with np.load(train_vectors) as npz:
    X_index_train = npz['X_index']
    Y_all_train   = npz['Y_all']
    Z_all_train   = npz['Z_all']

with np.load(test_vectors) as npz:
    X_index_test  = npz['X_index']
    Y_all_test    = npz['Y_all']
    Z_all_test    = npz['Z_all']

X_all_train = Z_all_train[X_index_train[:, 0], :]
X_all_test  = Z_all_test[X_index_test[:, 0],   :]

kmeans = pickle.load(open('kmeans.pickle', 'rb'))

clusters_train = kmeans.predict(Y_all_train - X_all_train)
clusters_test  = kmeans.predict(Y_all_test  - X_all_test)

In [33]:
model = MODELS[model_type](x_size=Z_all_train.shape[1], y_size=Y_all_train.shape[1], w_stddev=stddev, lambda_=lambdac)
print(model, flush=True)

for path in glob.glob('%s.k*.trained*' % model_type):
    print('Removing a stale file: "%s".' % path, flush=True)
    os.remove(path)

if os.path.isfile('%s.test.npz' % model_type):
    print('Removing a stale file: "%s".' % ('%s.test.npz' % model_type), flush=True)
    os.remove('%s.test.npz' % model_type)

Y_hat_test = {}

for cluster in range(kmeans.n_clusters):
    data = Data(
        cluster, clusters_train, clusters_test,
        X_index_train, Y_all_train, Z_all_train,
        X_index_test,  Y_all_test,  Z_all_test
    )

    saver = tf.train.Saver()
    saver_path = './%s.k%d.trained' % (model_type, cluster + 1)
    print (saver_path)
    Y_hat_test[str(cluster)] = train(model, data, callback=lambda sess: saver.save(sess, saver_path))
    print('Writing the output model to "%s".' % saver_path, flush=True)

test_path = '%s.test.npz' % model_type
np.savez_compressed(test_path, **Y_hat_test)
print('Writing the test data to "%s".' % test_path)


<RegularizedSynonymPhi lambda=0.100000>
./regularized_synonym_phi.k1.trained
Cluster 1: 72 train items and 12 test items available; using 1 steps of 72 items.


Cluster 1: epoch = 00001, train loss = 1.914667, test loss = 1.879742.
Cluster 1: epoch = 00010, train loss = 1.888259, test loss = 1.854691.
Cluster 1: epoch = 00020, train loss = 1.842801, test loss = 1.811686.
Cluster 1: epoch = 00030, train loss = 1.788292, test loss = 1.760276.
Cluster 1: epoch = 00040, train loss = 1.728872, test loss = 1.704432.
Cluster 1: epoch = 00050, train loss = 1.667302, test loss = 1.646803.
Cluster 1: epoch = 00060, train loss = 1.605547, test loss = 1.589269.
Cluster 1: epoch = 00070, train loss = 1.545025, test loss = 1.533183.
Cluster 1: epoch = 00080, train loss = 1.486731, test loss = 1.479491.
Cluster 1: epoch = 00090, train loss = 1.431339, test loss = 1.428822.
Cluster 1: epoch = 00100, train loss = 1.379262, test loss = 1.381560.
Cluster 1: epoch = 00110, train loss = 1.330715, test loss = 1.337893.
Cluster 1: epoch = 00120, train loss = 1.285763, test loss = 1.297864.
Cluster 1: epoch = 00130, train loss = 1.244363, test loss = 1.261413.
Cluste

Cluster 1 done in 0:00:01.371756.
Writing the output model to "./regularized_synonym_phi.k1.trained".
./regularized_synonym_phi.k2.trained
Cluster 2: 297 train items and 83 test items available; using 1 steps of 297 items.


Cluster 2: epoch = 00001, train loss = 1.921374, test loss = 1.899942.
Cluster 2: epoch = 00010, train loss = 1.795945, test loss = 1.777717.
Cluster 2: epoch = 00020, train loss = 1.611613, test loss = 1.598360.
Cluster 2: epoch = 00030, train loss = 1.429246, test loss = 1.421304.
Cluster 2: epoch = 00040, train loss = 1.270678, test loss = 1.267837.
Cluster 2: epoch = 00050, train loss = 1.143804, test loss = 1.145611.
Cluster 2: epoch = 00060, train loss = 1.048172, test loss = 1.054105.
Cluster 2: epoch = 00070, train loss = 0.978980, test loss = 0.988549.
Cluster 2: epoch = 00080, train loss = 0.929947, test loss = 0.942745.
Cluster 2: epoch = 00090, train loss = 0.895068, test loss = 0.910778.
Cluster 2: epoch = 00100, train loss = 0.869457, test loss = 0.887847.
Cluster 2: epoch = 00110, train loss = 0.849551, test loss = 0.870452.
Cluster 2: epoch = 00120, train loss = 0.832966, test loss = 0.856264.
Cluster 2: epoch = 00130, train loss = 0.818227, test loss = 0.843846.
Cluste

Cluster 2 done in 0:00:02.666215.
Writing the output model to "./regularized_synonym_phi.k2.trained".
./regularized_synonym_phi.k3.trained
Cluster 3: 100 train items and 20 test items available; using 1 steps of 100 items.


...

Cluster 23 done in 0:00:01.915170.
Writing the output model to "./regularized_synonym_phi.k23.trained".
./regularized_synonym_phi.k24.trained
Cluster 24: 166 train items and 48 test items available; using 1 steps of 166 items.


Cluster 24: epoch = 00001, train loss = 1.946589, test loss = 1.940476.
Cluster 24: epoch = 00010, train loss = 1.864942, test loss = 1.862423.
Cluster 24: epoch = 00020, train loss = 1.735578, test loss = 1.738824.
Cluster 24: epoch = 00030, train loss = 1.595357, test loss = 1.604890.
Cluster 24: epoch = 00040, train loss = 1.459518, test loss = 1.475167.
Cluster 24: epoch = 00050, train loss = 1.336435, test loss = 1.357674.
Cluster 24: epoch = 00060, train loss = 1.230015, test loss = 1.256171.
Cluster 24: epoch = 00070, train loss = 1.141164, test loss = 1.171560.
Cluster 24: epoch = 00080, train loss = 1.068904, test loss = 1.102941.
Cluster 24: epoch = 00090, train loss = 1.011225, test loss = 1.048423.
Cluster 24: epoch = 00100, train loss = 0.965691, test loss = 1.005704.
Cluster 24: epoch = 00110, train loss = 0.929847, test loss = 0.972450.
Cluster 24: epoch = 00120, train loss = 0.901453, test loss = 0.946526.
Cluster 24: epoch = 00130, train loss = 0.878603, test loss = 0.

Cluster 24 done in 0:00:02.011507.
Writing the output model to "./regularized_synonym_phi.k24.trained".
./regularized_synonym_phi.k25.trained
Cluster 25: 72 train items and 20 test items available; using 1 steps of 72 items.


Cluster 25: epoch = 00001, train loss = 1.856574, test loss = 1.797412.
Cluster 25: epoch = 00010, train loss = 1.826141, test loss = 1.767758.
Cluster 25: epoch = 00020, train loss = 1.774046, test loss = 1.717089.
Cluster 25: epoch = 00030, train loss = 1.711911, test loss = 1.656826.
Cluster 25: epoch = 00040, train loss = 1.644542, test loss = 1.591725.
Cluster 25: epoch = 00050, train loss = 1.575135, test loss = 1.524952.
Cluster 25: epoch = 00060, train loss = 1.505959, test loss = 1.458757.
Cluster 25: epoch = 00070, train loss = 1.438626, test loss = 1.394725.
Cluster 25: epoch = 00080, train loss = 1.374259, test loss = 1.333957.
Cluster 25: epoch = 00090, train loss = 1.313590, test loss = 1.277156.
Cluster 25: epoch = 00100, train loss = 1.257076, test loss = 1.224753.
Cluster 25: epoch = 00110, train loss = 1.204928, test loss = 1.176924.
Cluster 25: epoch = 00120, train loss = 1.157187, test loss = 1.133681.
Cluster 25: epoch = 00130, train loss = 1.113757, test loss = 1.

Cluster 25 done in 0:00:01.556090.
Writing the output model to "./regularized_synonym_phi.k25.trained".
Writing the test data to "regularized_synonym_phi.test.npz".


In [34]:
# move generated checkpoint files, 
if not w2v:
    directory = './ft-300-k%02d-l%.1f/' % (cluster+1, lambdac)
else:
    directory = './en_300-k%02d-l%.1f/' % (cluster+1, lambdac)

# create directory if it doesn't exist already
if not os.path.exists(directory):
    print ("Create directory %s" % (directory))
    os.makedirs(directory)

# move all interim checkpoints and final model to designated directory
output_files = glob.glob('%s.*.trained*'%(model_type))
if os.path.exists('%s.test.npz'%(model_type)):
    output_files.append('%s.test.npz'%(model_type))

for file in output_files:
    os.rename("./%s"%(file), "%s%s"%(directory, file))
    
# copy cluster pickle to directory as well
shutil.copy2('./kmeans.pickle', directory)

Create directory ./ft-300-k25-l0.1/


'./ft-300-k25-l0.1/kmeans.pickle'